In [24]:
import datetime
import time
from oauth2client.service_account import ServiceAccountCredentials
import gspread

days_ago = datetime.datetime.now() - datetime.timedelta(days=1)
yesterday= days_ago.strftime("%Y-%m-%d")
print(yesterday)


2021-06-20


In [25]:
days_ago = datetime.datetime.now() - datetime.timedelta(days=2)
yesterday2 = days_ago.strftime("%Y-%m-%d")
yesterday2

'2021-06-19'

In [26]:
days_ago = datetime.datetime.now() - datetime.timedelta(days=3)
yesterday3 = days_ago.strftime("%Y-%m-%d")
yesterday3

'2021-06-18'

In [27]:
time1 = datetime.datetime(2021, 6, 16, 9, 0, 0) #기준일 21/06/16
time2 = datetime.datetime.now()
date_gs = (time2-time1).days # 기준날짜와 오늘 날짜의 차
date_gs

5

In [28]:
weekday_name = ['월', '화', '수', '목', '금', '토', '일']
r = datetime.datetime.today().weekday()
weekday_name[r]

'월'

In [29]:
campaign_list = [6223, 6224]


In [30]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(chrome_options=options, executable_path=r'G:\내 드라이브\업무\dev\RPA\chromedriver.exe')
# driver.get("http://admin.mobiconnect.co.kr/admin/login")
# driver.quit()

Opera/9.80 (X11; Linux i686; U; ru) Presto/2.8.131 Version/11.11


C:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [31]:
driver.get('http://admin.mobiconnect.co.kr/admin/login')
driver.find_element_by_id('userId').send_keys('jmyang')
driver.find_element_by_id('userPwd').send_keys('mobi1109')
driver.find_element_by_xpath('//*[@id="frm"]/li[4]/input').click()

In [32]:
for i in campaign_list:
    campaign = i
    
    time.sleep(0.7)
    driver.get('http://admin.mobiconnect.co.kr/admin/adoffers/OfferDetail?offerId='+str(campaign)+'&startDate='+str(yesterday)+'&endDate='+str(yesterday))
    time.sleep(0.7)
    total_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[2]').text
    install_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[4]').text
    event_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[5]').text
    install_data=install_data[:-3]
    event_data = event_data[:-3]
    print(total_data)
    print(install_data)
    print(event_data)
    
    # 구글 시트에 입력 (kb)
    scope = ['https://spreadsheets.google.com/feeds']
    json_file_name = './rpadaily-a0b4f0f12ff6.json'
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
    gc = gspread.authorize(credentials)
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1TaRqRwAbH9NnGR0zl9cynj7l7nmOzSodmqo6HoKJVQA/edit#gid=2124347354'
    
    if campaign == 6224:
        doc = gc.open_by_url(spreadsheet_url)
        sheet1 = doc.worksheet('KBM_able_iOS')
    elif campaign== 6223:
        doc = gc.open_by_url(spreadsheet_url)
        sheet1 = doc.worksheet('KBM_able_AOS')
    else: 
        print("kb캠페인이 아닙니다")
        
    # 기준일(469+ 차이, but 매달 수정해줘야 함)
    sheet1.update_acell('c'+str(469+date_gs), total_data)
    sheet1.update_acell('d'+str(469+date_gs), install_data)
    sheet1.update_acell('e'+str(469+date_gs), event_data)    
    


2,071,317
3
1
2,692,903
1
0


In [33]:
if weekday_name[r] =='월':
    for i in campaign_list:
        campaign = i
        
        time.sleep(0.7)
        
        for j in [yesterday2, yesterday3]:
            driver.get('http://admin.mobiconnect.co.kr/admin/adoffers/OfferDetail?offerId='+str(campaign)+'&startDate='+str(j)+'&endDate='+str(j))
            time.sleep(0.7)
            total_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[2]').text
            install_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[4]').text
            event_data = driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]/tfoot/tr/td[5]').text
            install_data=install_data[:-3]
            event_data = event_data[:-3]
            print(total_data)
            print(install_data)
            print(event_data)
    
        # 구글 시트에 입력 (kb)
            scope = ['https://spreadsheets.google.com/feeds']
            json_file_name = './rpadaily-a0b4f0f12ff6.json'
            credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
            gc = gspread.authorize(credentials)
            spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1TaRqRwAbH9NnGR0zl9cynj7l7nmOzSodmqo6HoKJVQA/edit#gid=2124347354'
    
            if campaign == 6224:
                doc = gc.open_by_url(spreadsheet_url)
                sheet1 = doc.worksheet('KBM_able_iOS')
            elif campaign== 6223:
                doc = gc.open_by_url(spreadsheet_url)
                sheet1 = doc.worksheet('KBM_able_AOS')
            else: 
                print("kb캠페인이 아닙니다")
        
    # 기준일(469+ 차이, but 매달 수정해줘야 함)
            if j == yesterday2: #토욜
                sheet1.update_acell('c'+str(469+date_gs-1), total_data)
                sheet1.update_acell('d'+str(469+date_gs-1), install_data)
                sheet1.update_acell('e'+str(469+date_gs-1), event_data) 
            elif j == yesterday3: #금욜
                sheet1.update_acell('c'+str(469+date_gs-2), total_data)
                sheet1.update_acell('d'+str(469+date_gs-2), install_data)
                sheet1.update_acell('e'+str(469+date_gs-2), event_data) 
                    
    driver.quit()    

else:
    driver.quit()

1,493,630
1
1
1,817,799
2
0
3,002,716
2
1
3,714,297
3
1
